<a href="https://colab.research.google.com/github/Amit198181/JBVNL/blob/main/105__LTIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving LTIS_APR_AUG.xlsx to LTIS_APR_AUG.xlsx
User uploaded file "LTIS_APR_AUG.xlsx" with length 2230734 bytes


# Task
Analyze the "LTIS_APR_AUG.xlsx" file to determine if the maximum demand exceeded the contract demand by 5% or more at least four times during the last financial year.

## Load the data

### Subtask:
Load the `LTIS_APR_AUG.xlsx` file into a pandas DataFrame.


**Reasoning**:
The subtask is to load the Excel file into a pandas DataFrame. This involves importing the pandas library and using the `read_excel` function. Displaying the head and info of the DataFrame helps in understanding its structure and content, which is the next logical step after loading the data.



In [2]:
import pandas as pd

df = pd.read_excel('LTIS_APR_AUG.xlsx')
display(df.head())
display(df.info())

,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,LOAD_FACTOR_HTS_202506,SUPPLY_HOURS_HTS_202506,BILLED_UNITS_HTS_202507,MDI_HTS_202507,LOAD_FACTOR_HTS_202507,SUPPLY_HOURS_HTS_202507,BILLED_UNITS_HTS_202508,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508
0,RANCHI,RANCHI,RANCHI WEST,RT27482,11220279599,LIVE,M/S SABOO SONS PRATIK SABOO,LTIS,40.0,URBAN,...,0.0,0.0,95.0,12.0,0.0,0.0,275.0,18.0,0.0,0
1,RANCHI,RANCHI,RANCHI WEST,RT28138,11220279600,LIVE,SUVESH KUMAR,LTIS,8.0,RURAL,...,0.0,0.0,30.0,5.0,0.0,0.0,70.0,4.0,0.0,0
2,RANCHI,RANCHI,RANCHI WEST,RT27405,11220279612,LIVE,TAMIM ANSARI,LTIS,6.0,RURAL,...,0.0,0.0,61.0,6.0,0.0,0.0,120.0,5.0,0.0,0
3,RANCHI,RANCHI,RANCHI WEST,RT28556,11220279628,LIVE,"SARITA SRIVASTAVA, W/O - S K SRIVASTAVA,",LTIS,10.0,RURAL,...,0.0,0.0,55.0,4.0,0.0,0.0,150.0,6.0,0.0,0
4,RANCHI,RANCHI,RANCHI WEST,RT28602,11220279636,LIVE,SHWETA FABRICATION,LTIS,10.0,RURAL,...,0.0,0.0,25.0,5.0,0.0,0.0,102.0,6.0,0.0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17742 entries, 0 to 17741
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ZONE                     15715 non-null  object 
 1   CIRCLE                   15715 non-null  object 
 2   DIVISION                 15715 non-null  object 
 3   CONSNO                   15715 non-null  object 
 4   ACCOUNT_NO               17742 non-null  int64  
 5   CONSUMER_STATUS          15715 non-null  object 
 6   CONSUMER_NAME            15714 non-null  object 
 7   TARIFF_CODE              15715 non-null  object 
 8   CONTRACTED_LOAD          15715 non-null  float64
 9   CONSUMER_TYPE            15715 non-null  object 
 10  METER_READER_NAME        15703 non-null  object 
 11  MF_KWH                   15715 non-null  float64
 12  PREPAID_FLAG             15715 non-null  object 
 13  MTR_CONSUMP_TYPE         15715 non-null  object 
 14  BILLED_UNITS_HTS_20250

None

## Identify relevant columns

### Subtask:
Identify which columns contain the maximum demand and the contract demand.


**Reasoning**:
Based on the previous output from `df.head()` and `df.info()`, the column `CONTRACTED_LOAD` seems to represent the contract demand. The columns with the pattern `MDI_HTS_YYYYMM` appear to represent the maximum demand for different months. To confirm this and identify all relevant MDI columns, I will display the column names again.



In [3]:
print(df.columns.tolist())

['ZONE', 'CIRCLE', 'DIVISION', 'CONSNO', 'ACCOUNT_NO', 'CONSUMER_STATUS', 'CONSUMER_NAME', 'TARIFF_CODE', 'CONTRACTED_LOAD', 'CONSUMER_TYPE', 'METER_READER_NAME', 'MF_KWH', 'PREPAID_FLAG', 'MTR_CONSUMP_TYPE', 'BILLED_UNITS_HTS_202504', 'MDI_HTS_202504', 'BILLED_UNITS_HTS_202505', 'MDI_HTS_202505', 'BILLED_UNITS_HTS_202506', 'MDI_HTS_202506', 'LOAD_FACTOR_HTS_202506', 'SUPPLY_HOURS_HTS_202506', 'BILLED_UNITS_HTS_202507', 'MDI_HTS_202507', 'LOAD_FACTOR_HTS_202507', 'SUPPLY_HOURS_HTS_202507', 'BILLED_UNITS_HTS_202508', 'MDI_HTS_202508', 'LOAD_FACTOR_HTS_202508', 'SUPPLY_HOURS_HTS_202508']


## Calculate the threshold

### Subtask:
Calculate the 5% excess threshold for the contract demand.


**Reasoning**:
Calculate the 5% excess threshold for the contract demand by multiplying the `CONTRACTED_LOAD` column by 1.05 and store it in a new column named `CONTRACTED_LOAD_THRESHOLD`.



In [4]:
df['CONTRACTED_LOAD_THRESHOLD'] = df['CONTRACTED_LOAD'] * 1.05
display(df[['CONTRACTED_LOAD', 'CONTRACTED_LOAD_THRESHOLD']].head())

,CONTRACTED_LOAD,CONTRACTED_LOAD_THRESHOLD
0,40.0,42.0
1,8.0,8.4
2,6.0,6.3
3,10.0,10.5
4,10.0,10.5


## Compare demand to threshold

### Subtask:
Compare the maximum demand to the calculated threshold for each entry.


**Reasoning**:
Create boolean columns indicating if the maximum demand exceeded the calculated threshold for each month and display the relevant columns to verify the results.



In [5]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']
threshold_col = 'CONTRACTED_LOAD_THRESHOLD'

for col in demand_cols:
    df[f'EXCEEDED_THRESHOLD_{col[-6:]}'] = df[col] > df[threshold_col]

display(df[demand_cols + [threshold_col] + [f'EXCEEDED_THRESHOLD_{col[-6:]}' for col in demand_cols]].head())

,MDI_HTS_202504,MDI_HTS_202505,MDI_HTS_202506,MDI_HTS_202507,MDI_HTS_202508,CONTRACTED_LOAD_THRESHOLD,EXCEEDED_THRESHOLD_202504,EXCEEDED_THRESHOLD_202505,EXCEEDED_THRESHOLD_202506,EXCEEDED_THRESHOLD_202507,EXCEEDED_THRESHOLD_202508
0,12.0,12.0,12.0,12.0,18.0,42.0,False,False,False,False,False
1,3.2,5.0,5.0,5.0,4.0,8.4,False,False,False,False,False
2,6.2,4.0,5.0,6.0,5.0,6.3,False,False,False,False,False
3,3.3,3.0,3.0,4.0,6.0,10.5,False,False,False,False,False
4,4.4,3.0,4.0,5.0,6.0,10.5,False,False,False,False,False


## Count exceedances

### Subtask:
Count how many times the maximum demand exceeds the threshold.


**Reasoning**:
Select the threshold exceeded columns and sum the boolean values for each row to get the count of exceedances, then display the head of the dataframe with the new column.



In [6]:
exceeded_cols = [col for col in df.columns if 'EXCEEDED_THRESHOLD_' in col]
df['EXCEEDANCE_COUNT'] = df[exceeded_cols].sum(axis=1)
display(df.head())

,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508,CONTRACTED_LOAD_THRESHOLD,EXCEEDED_THRESHOLD_202504,EXCEEDED_THRESHOLD_202505,EXCEEDED_THRESHOLD_202506,EXCEEDED_THRESHOLD_202507,EXCEEDED_THRESHOLD_202508,EXCEEDANCE_COUNT
0,RANCHI,RANCHI,RANCHI WEST,RT27482,11220279599,LIVE,M/S SABOO SONS PRATIK SABOO,LTIS,40.0,URBAN,...,18.0,0.0,0,42.0,False,False,False,False,False,0
1,RANCHI,RANCHI,RANCHI WEST,RT28138,11220279600,LIVE,SUVESH KUMAR,LTIS,8.0,RURAL,...,4.0,0.0,0,8.4,False,False,False,False,False,0
2,RANCHI,RANCHI,RANCHI WEST,RT27405,11220279612,LIVE,TAMIM ANSARI,LTIS,6.0,RURAL,...,5.0,0.0,0,6.3,False,False,False,False,False,0
3,RANCHI,RANCHI,RANCHI WEST,RT28556,11220279628,LIVE,"SARITA SRIVASTAVA, W/O - S K SRIVASTAVA,",LTIS,10.0,RURAL,...,6.0,0.0,0,10.5,False,False,False,False,False,0
4,RANCHI,RANCHI,RANCHI WEST,RT28602,11220279636,LIVE,SHWETA FABRICATION,LTIS,10.0,RURAL,...,6.0,0.0,0,10.5,False,False,False,False,False,0


## Check the condition

### Subtask:
Determine if the count of exceedances is four or more.


**Reasoning**:
Filter the DataFrame to include only rows where the 'EXCEEDANCE_COUNT' is 4 or more and display the number of rows in the filtered DataFrame.



In [7]:
df_exceeding_threshold = df[df['EXCEEDANCE_COUNT'] >= 4].copy()
print(f"Number of entries with 4 or more exceedances: {len(df_exceeding_threshold)}")

Number of entries with 4 or more exceedances: 1757


## Report the result

### Subtask:
Provide a clear statement indicating whether the condition was met or not.


**Reasoning**:
Check if the df_exceeding_threshold DataFrame is empty and print the appropriate statement.



In [8]:
if not df_exceeding_threshold.empty:
    print(f"The condition was met for {len(df_exceeding_threshold)} entries.")
else:
    print("The condition was not met for any entry.")

The condition was met for 1757 entries.


## Summary:

### Data Analysis Key Findings

*   The analysis identified 1757 entries in the dataset where the maximum demand exceeded the contracted load by 5% or more in at least four of the recorded months (April 2025 to August 2025).
*   The threshold for excess demand was calculated as 105% of the `CONTRACTED_LOAD`.
*   Boolean indicators were created for each month to flag whether the maximum demand for that month exceeded the calculated threshold.
*   A count of these exceedances was calculated for each entry, and the analysis focused on entries with a count of 4 or more.

### Insights or Next Steps

*   Investigate the specific entries where the maximum demand significantly and repeatedly exceeded the contract demand. This could help identify consumers who might need to revise their contracts or face penalties.
*   Analyze the spatial distribution (`ZONE`, `CIRCLE`, `DIVISION`) or consumer types (`CONSUMER_TYPE`, `TARIFF_CODE`) of the entries with frequent exceedances to identify potential trends or areas for targeted intervention.


# Task
Calculate the enhanced contract demand for each consumer based on the average of the four highest monthly maximum demand readings from the uploaded CSV file.

## Identify relevant columns

### Subtask:
Identify the columns that contain the maximum demand readings for each month.


**Reasoning**:
Print the list of columns to identify the columns containing the maximum demand readings, which are expected to follow a pattern like `MDI_HTS_YYYYMM`.



In [9]:
print(df.columns.tolist())

['ZONE', 'CIRCLE', 'DIVISION', 'CONSNO', 'ACCOUNT_NO', 'CONSUMER_STATUS', 'CONSUMER_NAME', 'TARIFF_CODE', 'CONTRACTED_LOAD', 'CONSUMER_TYPE', 'METER_READER_NAME', 'MF_KWH', 'PREPAID_FLAG', 'MTR_CONSUMP_TYPE', 'BILLED_UNITS_HTS_202504', 'MDI_HTS_202504', 'BILLED_UNITS_HTS_202505', 'MDI_HTS_202505', 'BILLED_UNITS_HTS_202506', 'MDI_HTS_202506', 'LOAD_FACTOR_HTS_202506', 'SUPPLY_HOURS_HTS_202506', 'BILLED_UNITS_HTS_202507', 'MDI_HTS_202507', 'LOAD_FACTOR_HTS_202507', 'SUPPLY_HOURS_HTS_202507', 'BILLED_UNITS_HTS_202508', 'MDI_HTS_202508', 'LOAD_FACTOR_HTS_202508', 'SUPPLY_HOURS_HTS_202508', 'CONTRACTED_LOAD_THRESHOLD', 'EXCEEDED_THRESHOLD_202504', 'EXCEEDED_THRESHOLD_202505', 'EXCEEDED_THRESHOLD_202506', 'EXCEEDED_THRESHOLD_202507', 'EXCEEDED_THRESHOLD_202508', 'EXCEEDANCE_COUNT']


## Select the top four maximum demands

### Subtask:
For each consumer, select the four highest maximum demand values from the identified columns.


**Reasoning**:
Select the columns containing the monthly maximum demand values and apply a function to each row to get the top 4 maximum demand values, storing them in a new column.



In [10]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

def get_top_4_demand(row):
    return row[demand_cols].nlargest(4).tolist()

df['TOP_4_DEMANDS'] = df.apply(get_top_4_demand, axis=1)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS']].head())

TypeError: Cannot use method 'nlargest' with dtype object

**Reasoning**:
The error message indicates that 'nlargest' cannot be used with an object dtype. This suggests that one or more of the demand columns are not numerical. I need to convert the demand columns to a numeric type, coercing errors to NaN, before applying the `nlargest` function. Then, I will apply the function again to get the top 4 demand values for each row.



In [11]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

for col in demand_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

def get_top_4_demand(row):
    return row[demand_cols].nlargest(4).tolist()

df['TOP_4_DEMANDS'] = df.apply(get_top_4_demand, axis=1)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS']].head())

TypeError: Cannot use method 'nlargest' with dtype object

**Reasoning**:
The error still persists even after attempting to convert the columns to numeric. This suggests there might be persistent non-numeric values in the 'demand_cols'. I will inspect the data types of the columns again after conversion and also check for unique non-numeric values in those columns to understand the issue better.



In [12]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

for col in demand_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(df[demand_cols].dtypes)

for col in demand_cols:
    non_numeric_values = df[col][df[col].apply(lambda x: not pd.io.common.is_number(x) and pd.notnull(x))].unique()
    if len(non_numeric_values) > 0:
        print(f"Non-numeric values in {col}: {non_numeric_values}")

MDI_HTS_202504    float64
MDI_HTS_202505    float64
MDI_HTS_202506    float64
MDI_HTS_202507    float64
MDI_HTS_202508    float64
dtype: object


AttributeError: module 'pandas.io.common' has no attribute 'is_number'

**Reasoning**:
The previous attempt to check for non-numeric values failed due to an `AttributeError`. The traceback indicates that `pd.io.common.is_number` is not a valid attribute. Although the `dtypes` output shows float64, there might still be underlying issues with the data that `pd.to_numeric` with `errors='coerce'` didn't fully resolve for `nlargest`. A more robust way to check for non-numeric values after conversion is to check for non-finite values (NaN, inf, -inf). I will check for non-finite values in the demand columns and then re-attempt to calculate the top 4 demands.



In [13]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

for col in demand_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in demand_cols:
    non_finite_values = df[col][~np.isfinite(df[col])].unique()
    if len(non_finite_values) > 0:
        print(f"Non-finite values in {col}: {non_finite_values}")

def get_top_4_demand(row):
    return row[demand_cols].nlargest(4).tolist()

df['TOP_4_DEMANDS'] = df.apply(get_top_4_demand, axis=1)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS']].head())

NameError: name 'np' is not defined

**Reasoning**:
The previous code failed because the `numpy` library was not imported, resulting in a `NameError`. I need to import `numpy` to use `np.isfinite`. Then I can proceed with checking for non-finite values and calculating the top 4 demands.



In [14]:
import numpy as np

demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

for col in demand_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in demand_cols:
    non_finite_values = df[col][~np.isfinite(df[col])].unique()
    if len(non_finite_values) > 0:
        print(f"Non-finite values in {col}: {non_finite_values}")

def get_top_4_demand(row):
    return row[demand_cols].nlargest(4).tolist()

df['TOP_4_DEMANDS'] = df.apply(get_top_4_demand, axis=1)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS']].head())

Non-finite values in MDI_HTS_202504: [nan]
Non-finite values in MDI_HTS_202505: [nan]
Non-finite values in MDI_HTS_202506: [nan]
Non-finite values in MDI_HTS_202507: [nan]
Non-finite values in MDI_HTS_202508: [nan]


TypeError: Cannot use method 'nlargest' with dtype object

**Reasoning**:
Despite converting the columns to numeric and checking for non-finite values, the `TypeError` with `nlargest` persists. The error message still indicates a dtype object issue. This is unexpected given the `float64` dtypes printed. A possible reason is that the `apply` function is somehow changing the dtype within its scope, or there are some edge cases with NaN values and `nlargest` in this pandas version or environment. As a workaround, I will use a more explicit method to sort the values and select the top 4, which involves converting the row slice to a numpy array, sorting it, handling NaNs, and then selecting the top 4.



In [15]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

for col in demand_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

def get_top_4_demand_manual(row):
    demand_values = row[demand_cols].values
    # Sort in descending order, NaNs are placed at the end
    sorted_demand = np.sort(demand_values)[::-1]
    # Filter out NaNs and take the top 4
    top_4 = sorted_demand[np.isfinite(sorted_demand)][:4]
    return top_4.tolist()

df['TOP_4_DEMANDS'] = df.apply(get_top_4_demand_manual, axis=1)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS']].head())

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

**Reasoning**:
The error `TypeError: ufunc 'isfinite' not supported for the input types` indicates that the `demand_values` array contains elements that are not compatible with `np.isfinite`, even after using `pd.to_numeric(errors='coerce')`. This suggests there might be some persistent non-numeric or incompatible types in the data that are not being converted to `NaN`. I will inspect the unique values in the demand columns again, this time without assuming they are numbers after the initial conversion, to see if I can identify the problematic values.



In [16]:
demand_cols = ['MDI_HTS_202504', 'MDI_HTS_202505', 'MDI_HTS_202506', 'MDI_HTS_202507', 'MDI_HTS_202508']

for col in demand_cols:
    # Convert to numeric, coercing errors to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # Check for any values that are not finite numbers after coercion
    non_finite_mask = ~np.isfinite(df[col])
    non_finite_values = df.loc[non_finite_mask, col].unique()
    if len(non_finite_values) > 0:
        print(f"Non-finite values in {col}: {non_finite_values}")

def get_top_4_demand_robust(row):
    demand_values = row[demand_cols].values
    # Ensure values are numeric, converting any remaining non-numeric to NaN
    numeric_demand_values = pd.to_numeric(demand_values, errors='coerce')
    # Filter out NaNs and sort
    finite_demand = numeric_demand_values[np.isfinite(numeric_demand_values)]
    # Sort in descending order
    sorted_demand = np.sort(finite_demand)[::-1]
    # Take the top 4
    top_4 = sorted_demand[:4]
    # If there are less than 4 finite values, pad with NaNs or a specific value if required, but the instruction is to get the top 4
    # If there are fewer than 4 values, the slicing will just return fewer values, which is acceptable based on the instruction.
    return top_4.tolist()

df['TOP_4_DEMANDS'] = df.apply(get_top_4_demand_robust, axis=1)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS']].head())

Non-finite values in MDI_HTS_202504: [nan]
Non-finite values in MDI_HTS_202505: [nan]
Non-finite values in MDI_HTS_202506: [nan]
Non-finite values in MDI_HTS_202507: [nan]
Non-finite values in MDI_HTS_202508: [nan]


,ACCOUNT_NO,TOP_4_DEMANDS
0,11220279599,"[18.0, 12.0, 12.0, 12.0]"
1,11220279600,"[5.0, 5.0, 5.0, 4.0]"
2,11220279612,"[6.2, 6.0, 5.0, 5.0]"
3,11220279628,"[6.0, 4.0, 3.3, 3.0]"
4,11220279636,"[6.0, 5.0, 4.4, 4.0]"


## Calculate the average of the top four

### Subtask:
Calculate the average of these four highest maximum demand values for each consumer.


**Reasoning**:
Define a function to calculate the average of a list of numbers, handling non-finite values, and then apply this function to the `TOP_4_DEMANDS` column to create the `ENHANCED_CONTRACT_DEMAND` column.



In [17]:
import numpy as np

def calculate_average_top_4(demand_list):
    """Calculates the average of the top 4 demand values, ignoring non-finite values."""
    if not demand_list:
        return np.nan  # Return NaN for empty lists

    # Ensure values are numeric and filter out non-finite values
    finite_demand = [x for x in demand_list if np.isfinite(x)]

    if not finite_demand:
        return np.nan # Return NaN if no finite values are left

    return np.mean(finite_demand)

df['ENHANCED_CONTRACT_DEMAND'] = df['TOP_4_DEMANDS'].apply(calculate_average_top_4)

display(df[['ACCOUNT_NO', 'TOP_4_DEMANDS', 'ENHANCED_CONTRACT_DEMAND']].head())

,ACCOUNT_NO,TOP_4_DEMANDS,ENHANCED_CONTRACT_DEMAND
0,11220279599,"[18.0, 12.0, 12.0, 12.0]",13.500
1,11220279600,"[5.0, 5.0, 5.0, 4.0]",4.750
2,11220279612,"[6.2, 6.0, 5.0, 5.0]",5.550
3,11220279628,"[6.0, 4.0, 3.3, 3.0]",4.075
4,11220279636,"[6.0, 5.0, 4.4, 4.0]",4.850


## Display the result

### Subtask:
Display the relevant columns to show the calculated enhanced contract demand for each consumer.


**Reasoning**:
Display the specified columns to show the calculated enhanced contract demand.



In [18]:
display(df[['ACCOUNT_NO', 'CONTRACTED_LOAD', 'ENHANCED_CONTRACT_DEMAND']].head())

,ACCOUNT_NO,CONTRACTED_LOAD,ENHANCED_CONTRACT_DEMAND
0,11220279599,40.0,13.500
1,11220279600,8.0,4.750
2,11220279612,6.0,5.550
3,11220279628,10.0,4.075
4,11220279636,10.0,4.850


## Summary:

### Data Analysis Key Findings

*   The relevant columns for maximum demand readings were identified based on the pattern `MDI_HTS_YYYYMM`.
*   For each consumer, the top four highest maximum demand values were successfully selected from the identified monthly demand columns.
*   The average of the top four maximum demand values was calculated for each consumer and stored in a new column `ENHANCED_CONTRACT_DEMAND`.

### Insights or Next Steps

*   The calculated enhanced contract demand could be used to compare against the existing contracted load and identify potential discrepancies or opportunities for contract optimization.


In [19]:
# Define the filename for the Excel file
output_filename = 'analyzed_LTIS_data.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(output_filename, index=False)

# Download the file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>